<a href="https://colab.research.google.com/github/Luck1e23/STA160-Team-11-Project/blob/Tina/STA160_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from PIL import Image
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.optim import Adam

# Visualization tools
import torchvision
import torchvision.transforms.v2 as transforms
import torchvision.transforms.functional as F
import matplotlib.pyplot as plt

In [ ]:
torchvision.datasets.MNIST("./",download=True)

100%|██████████| 9.91M/9.91M [00:00<00:00, 57.4MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.71MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 14.9MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 9.95MB/s]


Dataset MNIST
    Number of datapoints: 60000
    Root location: ./
    Split: Train

In [3]:
# Create the Dataset class, NIHXrays, which contains the normalized pixel values of all the images

class NIHXrays(Dataset):
    def __init__(self, file_path, dataset_root, list_file = None):
        self.data = pd.read_csv(file_path)
        self.dataset_root = dataset_root

        if list_file:
            # Open the file and convert into a list of image names
            with open(list_file, 'r') as f:
                image_list = [line.strip() for line in f.readlines()]
            # Keep only rows that match the listed image names
            self.data = self.data[self.data['Image Index'].isin(image_list)].reset_index(drop = True)

        # Label map
        all_labels = set()
        for labels in self.data['Finding Labels']:
            for l in labels.split('|'):
                all_labels.add(l.strip())
        self.all_labels = sorted(list(all_labels))
        self.label_map = {label: idx for idx, label in enumerate(self.all_labels)}

        self.finding_labels = []
        for labels in self.data['Finding Labels']:
            vec = torch.zeros(len(self.all_labels))
            # Obtaining all the unique disease names
            for l in labels.split('|'):
                l = l.strip()
                if l in self.label_map:
                    vec[self.label_map[l]] = 1.0 # Mark as 1 if that is the disease found
            self.finding_labels.append(vec)

        # Image map
        self.image_map = {}
        # Go through all subfolders
        for root, dirs, files in os.walk(dataset_root):
            # Only keep 'images' folders
            if os.path.basename(root) == "images":
                for f in files:
                    self.image_map[f] = os.path.join(root, f)

        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor() # Converts image to tensor and normalizes pixel values
            ])

    def __len__(self):
        # Get total number of samples
        return len(self.data)

    def __getitem__(self, idx):

        row = self.data.iloc[idx]
        img_name = row['Image Index']
        finding_label = self.finding_labels[idx]

        # Load and transform the image
        path = self.image_map.get(img_name)
        if not path:
            raise FileNotFoundError(f"Image {img_name} not found in dataset root.")
        img = self.transform(Image.open(path).convert('L')) # Convert to grayscale, resize, and a tensor

        return img, finding_label, img_name

In [4]:
file_path = '/content/drive/Shareddrives/STA_160/dataset/Data_Entry_2017.csv'
dataset_root = '/content/drive/Shareddrives/STA_160/dataset'
train_val = '/content/drive/Shareddrives/STA_160/dataset/train_val_list.txt'
test = '/content/drive/Shareddrives/STA_160/dataset/test_list.txt'


train_val_data = NIHXrays(file_path, dataset_root, list_file = train_val)
test_data = NIHXrays(file_path, dataset_root, list_file = test)

/usr/local/lib/python3.12/dist-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [5]:
# Split the train_val_data into training and validation datasets
train_data, valid_data = torch.utils.data.random_split(train_val_data, [0.8, 0.2])

# Create DataLoaders for training and validation
n = 32
train_loader = DataLoader(train_data, batch_size=n, shuffle=True)
train_N = len(train_loader.dataset)
valid_loader = DataLoader(valid_data, batch_size=n)
valid_N = len(valid_loader.dataset)


# Data Augmentation
IMG_WIDTH, IMG_HEIGHT = (224, 224)
rand_transforms = transforms.Compose([
    transforms.RandomRotation(25),
    transforms.RandomResizedCrop((IMG_WIDTH, IMG_HEIGHT), scale = (0.8, 1), ratio = (1, 1)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2)
])